In [1]:
import pandas as pd
import datetime
pd.set_option('display.max_colwidth', 128)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 60)

In [2]:
"""
-- Modified original query from https://dune.com/queries/92408/184718

SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID,
  tx.value/1e18 AS price_eth,
  date_trunc('second', mints."evt_block_time") AS time,   
  mints."_to" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721_evt_Mint" mints -- old contrct
LEFT JOIN ethereum.transactions tx
  ON mints."evt_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"

UNION ALL 
    
SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID, 
  tx.value/1e18 AS price, 
  date_trunc('second', mints."call_block_time") AS time, 
  mints."_by" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721Core_call_mint" mints -- new contract
LEFT JOIN ethereum.transactions tx
  ON mints."call_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"
WHERE "output__tokenId" is not null
ORDER BY time DESC
"""

d = pd.read_csv('../mint.csv')
d["time"] = pd.to_datetime(d["time"])
display(d.dtypes)

# sort by time and descending gas for most probable execution order without looking at transaction order numbers.
d.sort_values(by=["time", "gas_eth"], ascending=[True, False], inplace=True)

d.head()

hash                      object
success                     bool
projectid                  int64
price_eth                float64
time         datetime64[ns, UTC]
buyer                     object
gas_eth                  float64
dtype: object

,hash,success,projectid,price_eth,time,buyer,gas_eth
204059,\xc86f9caf0307f66d63c03aa1952f47e1a7f09243a8e7e3b26faa374a60c253dc,True,2,0.10,2020-11-27 15:58:01+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015308
204058,\x2b4c7709bcb24f5f0337fcf5c045a5cbc911ab118f8bd0439db52615aa12d2ad,True,2,0.10,2020-11-27 16:00:31+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.013010
204057,\x99666b7a136f58b78abbb8226bae746e51293c927d2efc8d3d2e4bb4f3c1f500,True,2,0.10,2020-11-27 16:08:37+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015034
204056,\x148da1d93e382e3220df987557f0240f14e24772feb4435a7a0337d607da6cdf,True,1,0.05,2020-11-27 16:10:41+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.017108
204055,\x9e812dec2467b3f9c84fc3e589cc608360284a51def0bcae675fd43bae4da26f,True,2,0.10,2020-11-27 16:11:28+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.014986


In [3]:
display(d.info())
display("Number of successful mints: ", d.success.sum())
d.describe(include=['bool','float', 'int', 'datetime'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204060 entries, 204059 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   hash       204060 non-null  object             
 1   success    204060 non-null  bool               
 2   projectid  204060 non-null  int64              
 3   price_eth  204060 non-null  float64            
 4   time       204060 non-null  datetime64[ns, UTC]
 5   buyer      204060 non-null  object             
 6   gas_eth    204060 non-null  float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(2), int64(1), object(2)
memory usage: 11.1+ MB


None

'Number of successful mints: '

198604

,success,projectid,price_eth,gas_eth
count,204060,204060.000000,204060.000000,204060.000000
unique,2,NaN,NaN,NaN
top,True,NaN,NaN,NaN
freq,198604,NaN,NaN,NaN
mean,NaN,152.841855,0.460313,0.074256
std,NaN,98.342955,0.942731,0.142770
min,NaN,0.000000,0.000000,0.000000
25%,NaN,74.000000,0.100000,0.016525
50%,NaN,143.000000,0.120000,0.038102
75%,NaN,227.000000,0.287963,0.074728


In [27]:
mints = d[d["success"]]

import statistics

def getMiddleValue(pdSeries):
    mid = (pdSeries.count() / 2).astype(int)
    return pdSeries.iloc[mid]

mintsByProjectId = mints.groupby("projectid")
mintsByProjectId = pd.DataFrame({
    "count": mintsByProjectId["projectid"].count(),
    "firstMintTime": mintsByProjectId["time"].first(),
    "lastMintTime": mintsByProjectId["time"].last(),
    "lastMintPriceTotal": mintsByProjectId["price_eth"].last() + mintsByProjectId["gas_eth"].last(),
	"minMintPrice": mintsByProjectId["price_eth"].min(),
	"medianMintPrice": mintsByProjectId["price_eth"].median(),
	"meanMintPrice": mintsByProjectId["price_eth"].mean(),
    "medianMintTime":  mintsByProjectId["time"].apply(lambda x: getMiddleValue(x))
    })
mintsByProjectId["latterMintWindowInMins"] = round((mintsByProjectId["lastMintTime"] - mintsByProjectId["medianMintTime"]).dt.total_seconds() / 60, 2)
mintsByProjectId["totalMintWindowInMins"] = ((mintsByProjectId["lastMintTime"] - mintsByProjectId["firstMintTime"]).dt.total_seconds() / 60)
mintsByProjectId["mintWindowUnder4Hours"] = (mintsByProjectId["totalMintWindowInMins"] <= 240)
mintsByProjectId = mintsByProjectId.reset_index()

In [29]:
from ipywidgets import *

settings = ["medianMintPrice", "meanMintPrice", "minMintPrice", "totalMintWindowInMins"]
def update(i = 1):
	mintsByProjectId.plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="all collections")
	mintsByProjectId[mintsByProjectId["mintWindowUnder4Hours"] == True].plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="sold out under 4 hours")

interact(update)

interactive(children=(IntSlider(value=1, description='i', max=3, min=-1), Output()), _dom_classes=('widget-int…

<function __main__.update(i=1)>

In [6]:
"""
-- Modified original query from https://dune.com/queries/160701/314169

select distinct block_time, 
  ROUND("nft_token_id"::numeric / 1000000) as projectid,
  round(eth_amount, 2) as eth_price, 
  usd_price, 
  link, 
  platform, 
  left(seller::text, 7) as seller, 
  left(buyer::text, 7) as buyer 
from 
(
select 
  block_time, 
  platform, 
  usd_amount, 
     
  case 
     when ("original_currency" = 'ETH' OR "original_currency" = 'WETH')
             THEN  ("original_amount")
    else 0  
  END as eth_amount, 
  "usd_amount" as usd_price,

   
 CONCAT('<a href="https://opensea.io/assets/', CONCAT('0x', substring(a."nft_contract_address"::text from 3)), '/', a.nft_token_id,  '/?ref=0x8F903cFC0Af3C2EC0d872c57538AF5e071544a57','" target="_blank" >', 'View on OS', '</a>') as  link,
   
 seller, 
 buyer, 
 tx_hash,
 nft_token_id

from nft."trades" a
WHERE 
     "trade_type" = 'Single Item Trade'
     AND (a.nft_contract_address = '\xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270'
    OR  a.nft_contract_address = '\x059edd72cd353df5106d2b9cc5ab83a52287ac3a')
ORDER BY block_time DESC 
) gg
-- WHERE block_time > '{{Date}}'
order by block_time DESC
"""

p = pd.read_csv('../sales.csv')

display("Before filtering:", len(d))

# cleaning up weird project ids
p_removed = p[p["projectid"].str.len() >= 8]
p = p[p["projectid"].str.len() < 8]

# casting
p["block_time"] = pd.to_datetime(p["block_time"])
p["projectid"] = p["projectid"].astype(int)

p.sort_values(by=["block_time"], ascending=[True], inplace=True)


# adding derived data
p["normalized_price"] = p["eth_price"] / p["projectid"].map(mintsByProjectId.lastMintPriceTotal)
p["lastMintTime"] = p["projectid"].map(mintsByProjectId.lastMintTime)
p["isWithin2hFromLastMintTime"] = ((p["block_time"] - p["lastMintTime"]).dt.total_seconds() / 60 < 120) & ((p["block_time"] - p["lastMintTime"]).dt.total_seconds() > 0)

display(p.dtypes)
display(p.describe(include=['bool','float', 'int', 'datetime']))
p

/home/user42/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


'Before filtering:'

204060

block_time                    datetime64[ns, UTC]
projectid                                   int64
eth_price                                 float64
usd_price                                 float64
link                                       object
platform                                   object
seller                                     object
buyer                                      object
normalized_price                          float64
lastMintTime                  datetime64[ns, UTC]
isWithin2hFromLastMintTime                   bool
dtype: object

,projectid,eth_price,usd_price,normalized_price,isWithin2hFromLastMintTime
count,43833.000000,43833.00000,4.376100e+04,42890.000000,43833
unique,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,43792
mean,204.691146,1.57886,4.685512e+03,8.794666,NaN
std,84.607758,5.71663,1.843462e+04,33.947967,NaN
min,0.000000,0.00000,0.000000e+00,0.000000,NaN
25%,157.000000,0.17000,4.031410e+02,0.606368,NaN
50%,215.000000,0.48000,1.281197e+03,2.143739,NaN
75%,253.000000,1.40000,4.044180e+03,6.574600,NaN


,block_time,projectid,eth_price,usd_price,link,platform,seller,buyer,normalized_price,lastMintTime,isWithin2hFromLastMintTime
65535,2021-10-31 16:20:04+00:00,188,0.75,3145.33500,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/188000723/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\x81a4a,\xa5430,5.943664,2022-08-06 20:09:24+00:00,False
65534,2021-10-31 16:45:39+00:00,156,0.09,380.65320,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/156000101/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\x5b873,\xe0d66,0.041576,2021-09-13 16:41:40+00:00,False
65533,2021-10-31 16:52:02+00:00,182,0.10,421.89500,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/182000089/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\xc7cf7,\xe0d66,0.480166,2022-08-12 14:18:33+00:00,False
65532,2021-10-31 17:06:48+00:00,188,0.35,1485.52600,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/188000612/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\xbb5bc,\xf6a6e,2.773710,2022-08-06 20:09:24+00:00,False
65531,2021-10-31 17:09:44+00:00,153,0.12,492.34576,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/153000392/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\xb6d6b,\xa05f7,0.538261,2021-09-14 19:10:49+00:00,False
...,...,...,...,...,...,...,...,...,...,...,...
4,2022-08-13 06:51:34+00:00,116,0.06,110.28435,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/116003757/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\x62161,\x99500,0.355814,2021-08-02 17:06:52+00:00,False
3,2022-08-13 06:53:41+00:00,242,0.13,252.65142,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/242000228/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\x6c7f5,\x5c46a,1.652624,2022-01-28 17:25:56+00:00,False
2,2022-08-13 06:56:04+00:00,337,0.19,376.62040,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/337000497/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\x3615e,\x457ee,1.415163,2022-08-11 21:32:07+00:00,False
1,2022-08-13 06:57:34+00:00,116,0.07,140.23100,"<a href=""https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/116003379/?ref=0x8F903cFC0Af3C2EC0d872c57538AF...",OpenSea,\xed595,\x99500,0.415117,2021-08-02 17:06:52+00:00,False


In [7]:
tradesByProjectId = p.groupby("projectid")
tradesByProjectId = pd.DataFrame({
    "tradeCount": tradesByProjectId["projectid"].count(),
    "tradeCount2hr": tradesByProjectId["isWithin2hFromLastMintTime"].sum(),
    "medianNormPrice2h": tradesByProjectId.apply(lambda df: df[df["isWithin2hFromLastMintTime"]].normalized_price.median()),
})

In [8]:
byProjectId = pd.merge(mintsByProjectId, tradesByProjectId, on="projectid", how="outer")
d

,hash,success,projectid,price_eth,time,buyer,gas_eth
204059,\xc86f9caf0307f66d63c03aa1952f47e1a7f09243a8e7e3b26faa374a60c253dc,True,2,0.100000,2020-11-27 15:58:01+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015308
204058,\x2b4c7709bcb24f5f0337fcf5c045a5cbc911ab118f8bd0439db52615aa12d2ad,True,2,0.100000,2020-11-27 16:00:31+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.013010
204057,\x99666b7a136f58b78abbb8226bae746e51293c927d2efc8d3d2e4bb4f3c1f500,True,2,0.100000,2020-11-27 16:08:37+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015034
204056,\x148da1d93e382e3220df987557f0240f14e24772feb4435a7a0337d607da6cdf,True,1,0.050000,2020-11-27 16:10:41+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.017108
204055,\x9e812dec2467b3f9c84fc3e589cc608360284a51def0bcae675fd43bae4da26f,True,2,0.100000,2020-11-27 16:11:28+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.014986
...,...,...,...,...,...,...,...
14,\xc28bfb41b3db92ebc60b7a5840ae5d5fea79748086aca74a61f7b3c5409bd558,False,343,0.104868,2022-08-12 17:40:36+00:00,\x159d7bf20136eb23991bb4b0dfa974ca2051639a,0.000774
2,\xb2e923f375a724ae9411c106b9f3cac2482429ca48753b81859432254f070b8a,False,343,0.100000,2022-08-12 17:55:19+00:00,\x886478d3cf9581b624cb35b5446693fc8a58b787,0.000856
3,\xd89bb1b2bd909b8bb9db75d81250ef1090054997b52b00ca1dfce3c5426c82a6,False,343,0.104868,2022-08-12 17:55:19+00:00,\x886478d3cf9581b624cb35b5446693fc8a58b787,0.000801
1,\x840cd145bc54c011afa147afe590bb590e1e3d96be0c845a3c8acc0ef220d94b,True,297,0.090000,2022-08-12 21:06:01+00:00,\x55893ddb9a66ab306172e2fdc2723410598d5753,0.009956


In [9]:
from ipywidgets import *

def update(projectid = 331):
	min = d.groupby('projectid').min()
	pId = p[p["projectid"] == projectid]
	pId[pId["isWithin2hFromLastMintTime"]].plot(x="block_time", y=["normalized_price", "eth_price"], figsize=(20,8))

interact(update)

interactive(children=(IntSlider(value=331, description='projectid', max=993, min=-331), Output()), _dom_classe…

<function __main__.update(projectid=331)>

In [10]:
view = mints[mints["projectid"] == 337]